In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Importing all the libraries necessary for the project
import os
import sys
import time
import math
import random
import librosa
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.init as init
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from torch.utils.data import Dataset, random_split, DataLoader
import gc
from tqdm import tqdm, tqdm_notebook; tqdm.pandas()
import jax
import torchvision
import optax
import flax.linen as nn
import jax.nn
import jax.numpy as jnp
from flax import linen as nn
from tensorflow.keras.utils import to_categorical
seed = 1234
np.random.seed(seed)
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
warnings.filterwarnings("ignore")
from torchvision import transforms
import torch
from typing import Any
import functools
from flax.training import train_state
# to suppress warnings caused by cuda version
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from hmmlearn import hmm

In [2]:
jax.devices()

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [4]:
ROOT = "/kaggle/input/birdsong-recognition/"
os.listdir(ROOT)

In [5]:
df = pd.read_csv(os.path.join(ROOT, 'train.csv'))[['ebird_code', 'filename', 'duration']]
df['path'] = ROOT+'train_audio/' + df['ebird_code'] + "/" + df['filename']
df.head()

In [7]:
SEED = 42
FRAC = 0.2     # Validation fraction
SR = 44100     # sampling rate
MAXLEN= 60    # seconds
N_MELS = 128

seed_everything(SEED)
device = torch.device('cpu')

#Random sample of 15 birds
classes = set(random.sample(df['ebird_code'].unique().tolist(), 10)) 
print(classes)

In [8]:
df = df[df.ebird_code.apply(lambda x: x in classes)].reset_index(drop=True)
keys = set(df.ebird_code)
values = np.arange(0, len(keys))
code_dict = dict(zip(sorted(keys), values))
df['label'] = df['ebird_code'].apply(lambda x: code_dict[x])
df.head()

In [9]:
class BirdSoundDataset(Dataset):
    """Bird Sound dataset."""

    def __init__(self, df, transform = None):
        """
        Args:
            df (pd.DataFrame): must have ['path', 'label'] columns
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    
    def loadMP3(self, path, duration):
        """
        Args:
            path: path of the audio file 
        Returns:
            mels: Melspectrogram of the given audio file 
        """
        try:
            duration=5
            samples = SR* duration
            audio, _ = librosa.load(path, sr=SR)
            if 0 < len(audio):
                audio, _ = librosa.effects.trim(audio)
            if len(audio) > samples: # long enough
                audio = audio[0:0+samples]
            else: # pad blank
                padding = samples - len(audio)
                offset = padding // 2
                y = np.pad(audio, (offset, samples - len(audio) - offset), 'constant')

            mels = librosa.feature.melspectrogram(y=audio, sr=SR,n_mels=N_MELS, hop_length = 347,n_fft = N_MELS *20,fmin = 20, fmax = SR//2)
            mels = librosa.power_to_db(mels).astype(np.float32)
            mels = mels.transpose()
            eps = 0.001
            if np.std(mels) != 0:
                mels = (mels - np.mean(mels)) / np.std(mels)
            else:
                mels = (mels - np.mean(mels)) / eps
            return mels
            
        except Exception as e:
            print("Error encountered while parsing file: ", path, e)
            mels = np.zeros((N_MELS, MAXLEN*SR//347), dtype=np.float32)
            return mels
            
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        path = self.df['path'].iloc[idx]
    
        duration=5
        if os.path.exists("./"+path.split('/')[-1]+".npy"):
            mels = np.load("./"+path.split('/')[-1]+".npy")
        else:
            
            mels = self.loadMP3(path, duration)
            np.save("./"+path.split('/')[-1]+".npy", mels)
        label  = self.df['label'].iloc[idx]
        mels = np.resize(mels,(636,128,1))
        return mels, label


In [10]:
df = df.sample(frac=1).reset_index(drop=True)
train_len = int(len(df) * (1-FRAC))
train_df = df.iloc[:train_len]
valid_df = df.iloc[train_len:]
train_df.shape, valid_df.shape

In [11]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(BirdSoundDataset(train_df),
                                           batch_size=BATCH_SIZE, 
                                           num_workers=0, 
                                           shuffle=True, 
                                           drop_last = True)

valid_loader = torch.utils.data.DataLoader(BirdSoundDataset(valid_df), 
                                           batch_size=BATCH_SIZE, 
                                           num_workers=0, 
                                           shuffle=True, 
                                           drop_last = True)

len(train_loader), len(valid_loader)

In [12]:
(image_batch, label_batch) = next(iter(train_loader))
print(image_batch.shape)
print(label_batch.shape)

In [13]:
NUM_TPUS = jax.device_count()

def copy_dataset_to_devices(dataset, devices, num_reps=1):
    sharded_images = []
    sharded_labels = []
    for _ in range(num_reps):
        for image_batch, label_batch in tqdm(dataset, ncols=100):
            image_batch = image_batch.detach().cpu().numpy()
            image_batches = np.split(image_batch, NUM_TPUS, axis = 0)
            sharded_device_images = jax.device_put_sharded(image_batches, devices)
            sharded_images.append(sharded_device_images)

            label_batch = label_batch.detach().cpu().numpy()
            label_batches = np.split(label_batch, NUM_TPUS, axis = 0)
            sharded_device_labels = jax.device_put_sharded(label_batches, devices)
            sharded_labels.append(sharded_device_labels)

    return sharded_images, sharded_labels

devices = jax.local_devices()
print("NUM_TPUS:",NUM_TPUS)
sharded_training_images, sharded_training_labels = copy_dataset_to_devices(train_loader, devices, num_reps=10)

In [14]:
NUM_CLASSES = 10
class VGG19(nn.Module):
    @nn.compact
    def __call__(self, x, training):
        x = self._stack(x, 64, training)
        x = self._stack(x, 64, training)
        x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))
    
        x = self._stack(x, 128, training)
        x = self._stack(x, 128, training)
        x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))

        x = self._stack(x, 256, training)
        x = self._stack(x, 256, training)
        x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))    

        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))    
    
        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = self._stack(x, 512, training)
        x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))  
        x = x.reshape((x.shape[0], -1))

        x = nn.Dense(features=4096)(x)
        x = nn.BatchNorm(use_running_average=not training)(x)
        x = nn.relu(x)
        x = nn.Dropout(0.5, deterministic=not training)(x)

        x = nn.Dense(features=4096)(x)
        x = nn.BatchNorm(use_running_average=not training)(x)
        x = nn.relu(x)
        x = nn.Dropout(0.5, deterministic=not training)(x)
    
        x = nn.Dense(features=NUM_CLASSES)(x)
        return x
  
    @staticmethod
    def _stack(x, features, training, dropout=None):
        x = nn.Conv(features=features, kernel_size=(3, 3), padding='SAME')(x)
        x = nn.BatchNorm(use_running_average=not training)(x)
        x = nn.relu(x)
        return x

In [15]:
def average_metrics(metrics):
    '''
    Takes the list of dictionaries of the form k: v, and returns a dictionary
     of the form k: (average of the v).
    '''
    return {k: np.mean([metric[k] for metric in metrics])
        for k in metrics[0]}

In [16]:
def train(initial_network_state, num_epochs):
    '''
    Training the model from the given state, returns the state along with the training accuracies
    '''
    training_accuracies = []
    state = initial_network_state
    for i in range(num_epochs):
        batch_metrics = []
        for (image_batch, label_batch) in tqdm(zip(sharded_training_images,
                                               sharded_training_labels),
                                           total=len(sharded_training_images),
                                           ncols=100):
            state, metrics = train_batch(state, image_batch, label_batch)
            batch_metrics.append(metrics)
        train_metrics = average_metrics(batch_metrics)
        print(f'Epoch {i+1} done.', flush=True)
        print(f'  Loss: {train_metrics["loss"]:.4f}, '
          + f'accuracy: {train_metrics["accuracy"]:.4f}', flush=True)
        training_accuracies.append(train_metrics["accuracy"])
    return state, training_accuracies

In [17]:
class VGGState(train_state.TrainState):
    rng: Any
    batch_stats: Any
  
    @classmethod
    def create(cls, apply_fn, params, tx, rng, batch_stats):
        opt_state = tx.init(params)
        state = cls(0, apply_fn, params, tx, opt_state, rng, batch_stats)
        return state
  
    @classmethod
    def update_rng(cls, state, rng):
        return VGGState.create(state.apply_fn, state.params, state.tx, rng,
                           state.batch_stats)
  
    @classmethod
    def update_batch_stats(cls, state, batch_stats):
        return VGGState.create(state.apply_fn, state.params, state.tx,
                           state.rng, batch_stats)


In [18]:
def accuracy(logits, labels):
    '''
    Calcualtes the accuracy using the given logits and labels
    '''
    return jnp.mean(jnp.argmax(logits, -1) == labels)

def cross_entropy(logits, labels):
    '''
    Cross Entropy error between the logits and labels
    '''
    one_hot_labels = jax.nn.one_hot(labels, NUM_CLASSES)
    cross_entropy = optax.softmax_cross_entropy(logits, one_hot_labels)
    return jnp.mean(cross_entropy)

def training_loss(image_batch, label_batch, rng, batch_stats, params):
    '''
    Calculates the training loss 
    '''
    logits, batch_stats = VGG19().apply({'params': params,
                                       'batch_stats': batch_stats},
                                      image_batch, 
                                      training=True,
                                      rngs={'dropout': rng},
                                      mutable=['batch_stats'])
    loss = cross_entropy(logits, label_batch)
    return loss, (logits, batch_stats)

In [19]:
@functools.partial(jax.pmap, axis_name='tpu')
def train_batch(state, image_batch, label_batch):
    '''
    Training a single batch and returns loss and the accuracy
    '''
    rng, subrng = jax.random.split(state.rng)
    batch_loss_fn = functools.partial(training_loss, image_batch, label_batch,
                                    subrng, state.batch_stats)
    (batch_loss, (logits, batch_stats)), grads = \
    jax.value_and_grad(batch_loss_fn, has_aux=True)(state.params)
  
    gradsum = jax.lax.psum(grads, axis_name='tpu')

    state = state.apply_gradients(grads=gradsum)
    state = state.update_batch_stats(state, batch_stats['batch_stats'])
    state = state.update_rng(state, rng)

    batch_accuracy = accuracy(logits=logits, labels=label_batch)
    batch_accuracy_sum = jax.lax.pmean(batch_accuracy, axis_name='tpu')
    batch_loss = jax.lax.psum(batch_loss, axis_name='tpu')
    stats = {'loss': batch_loss, 'accuracy': batch_accuracy_sum}  

    return state, stats

In [20]:
def create_train_state(rng, dummy_image_batch):
    net = VGG19()
    params = net.init({'params': rng, 'dropout': rng}, dummy_image_batch, True)
    tx = optax.adam(learning_rate=0.01)
    state = VGGState.create(net.apply, params['params'], tx, rng,
                          params['batch_stats'])
    return state

In [21]:
rng = jax.random.PRNGKey(42)
rngs = np.broadcast_to(rng, (NUM_TPUS,) + rng.shape)
some_dummy_image_batch = sharded_training_images[0]
state = jax.pmap(create_train_state, axis_name='gpu')(rngs,some_dummy_image_batch)


In [22]:
start = time.time()
final_state, training_accuracies = train(state, num_epochs=25)
print("Total time: ", time.time() - start, "seconds")


In [23]:
plt.plot(training_accuracies)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:


# Class to handle all HMM related processing
class HMMTrainer(object):
    def __init__(self, model_name='GaussianHMM', n_components=4, cov_type='diag', n_iter=1000):
        self.model_name = model_name
        self.n_components = n_components
        self.cov_type = cov_type
        self.n_iter = n_iter
        self.models = []

        if self.model_name == 'GaussianHMM':
            self.model = hmm.GaussianHMM(n_components=self.n_components,
                                         covariance_type=self.cov_type, n_iter=self.n_iter)
        else:
            raise TypeError('Invalid model type')

    # X is a 2D numpy array where each row is 13D
    def train(self, X):
        np.seterr(all='ignore')
        self.models.append(self.model.fit(X))

    # Run the model on input data
    def get_score(self, input_data):
        return self.model.score(input_data)

In [ ]:
class SigProc():
    def __init__(self,filename):
        self.filename = filename
        self.signal, self.sr = librosa.load(filename)
        self.logMMSE = self.LogMMSE()
        self.MFCCs = self.MFCC()

    def LogMMSE(self):
        x = self.signal
        Slen = int(np.floor(0.02 * self.sr)) - 1
        noise_frames = 6

        PERC = 50
        len1 = int(np.floor(Slen * PERC / 100))
        len2 = Slen - len1

        win = np.hanning(Slen)
        win = win * len2 / np.sum(win)
        nFFT = 2 * Slen

        x_old = np.zeros(len1)
        Xk_prev = np.zeros(len1)
        Nframes = int(np.floor(len(x) / len2) - np.floor(Slen / len2))
        xfinal = np.zeros(Nframes * len2)

        noise_mean = np.zeros(nFFT)
        for j in range(0, Slen * noise_frames, Slen):
            noise_mean = noise_mean + np.absolute(np.fft.fft(win * x[j:j + Slen], nFFT, axis=0))
        noise_mu2 = noise_mean / noise_frames ** 2

        aa = 0.98
        mu = 0.98
        eta = 0.15
        ksi_min = 10 ** (-25 / 10)

        for k in range(0, Nframes * len2, len2):
            insign = win * x[k:k + Slen]

            spec = np.fft.fft(insign, nFFT, axis=0)
            sig = np.absolute(spec)
            sig2 = sig ** 2

            gammak = np.minimum(sig2 / noise_mu2, 40)

            if Xk_prev.all() == 0:
                ksi = aa + (1 - aa) * np.maximum(gammak - 1, 0)
            else:
                ksi = aa * Xk_prev / noise_mu2 + (1 - aa) * np.maximum(gammak - 1, 0)
                ksi = np.maximum(ksi_min, ksi)

            log_sigma_k = gammak * ksi / (1 + ksi) - np.log(1 + ksi)
            vad_decision = np.sum(log_sigma_k) / Slen
            if (vad_decision < eta):
                noise_mu2 = mu * noise_mu2 + (1 - mu) * sig2

            A = ksi / (1 + ksi)
            vk = A * gammak
            ei_vk = 0.5 * scipy.special.expn(1, vk)
            hw = A * np.exp(ei_vk)

            sig = sig * hw
            Xk_prev = sig ** 2
            xi_w = np.fft.ifft(hw * spec, nFFT, axis=0)
            xi_w = np.real(xi_w)

            xfinal[k:k + len2] = x_old + xi_w[0:len1]
            x_old = xi_w[len1:Slen]

        if not np.isnan(xfinal[0]):
            return xfinal
        else:
            return x

    def MFCC(self):
        MFCC = librosa.feature.mfcc(y=self.logMMSE, sr=self.sr, n_mfcc=40)
        return MFCC

def PlotImg(signal,sr,logMMSE,MFCCs):
    plt.style.use('seaborn-darkgrid')
    plt.rcParams['figure.figsize'] = (7, 6)

    plt.subplot(311)
    librosa.display.waveplot(signal, sr=sr)
    plt.title('Raw Wave')

    plt.subplot(312)
    librosa.display.waveplot(logMMSE, sr=sr)
    plt.title('MMSE-LSA')

    plt.subplot(313)
    librosa.display.specshow(MFCCs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    img = 'waveforms/{}.png'.format(int(time.time()))
    plt.savefig('/app/'+img)
    plt.close()
    return img

In [ ]:
from HMM import *
from tqdm import tqdm
from sigproc import SigProc
from sklearn.externals import joblib

def train_wavs(data_folder):
    hmm_models = []
    test_set = []
    for dirname in os.listdir(data_folder):
        # Get the name of the subfolder
        subfolder = os.path.join(data_folder, dirname)
        if not os.path.isdir(subfolder):
            continue
        # Extract the label
        label = subfolder[subfolder.rfind('/') + 1:]
        print('[*] '+label)
        # Initialize variables
        X = np.array([])
        y_words = []

        # Iterate through the audio files (leaving 1 file for testing in each class)
        files = [x for x in os.listdir(subfolder) if x.endswith('.wav')]
        #train_set = files[:-2]
        #test_set.append([os.path.join(subfolder,i) for i in files[-2:]])
        for filename in tqdm(train_set):
            # Extract Feature
            print 
            filepath = os.path.join(subfolder,filename)
            try:
                mfcc_features = SigProc(filepath).MFCC().T
            except:
                pass
            # Append to the variable X
            if len(X) == 0:
                X = mfcc_features
            else:
                X = np.append(X, mfcc_features, axis=0)

            # Append the label
            y_words.append(label)

        # Train and save HMM model
        hmm_trainer = HMMTrainer()
        hmm_trainer.train(X)
        hmm_models.append((hmm_trainer, label))
        joblib.dump(hmm_trainer, subfolder + "/ModelTrained.pkl")
        hmm_trainer = None
    return test_set

def recognize(mfcc_features):
    # Load Models
    hmm_models = []

    data_folder = '/app/data'
    for dirname in os.listdir(data_folder):
        # Get the name of the subfolder
        subfolder = os.path.join(data_folder, dirname)
        if not os.path.isdir(subfolder):
            continue
        # Extract the label
        label = subfolder[subfolder.rfind('/') + 1:]
        hmm_model = joblib.load(subfolder + "/ModelTrained.pkl")
        hmm_models.append((hmm_model, label))


    # Define variables
    max_score = -10000000000000.0
    output_label = None

    # Iterate through all HMM models and pick
    # the one with the highest score
    for item in hmm_models:
        hmm_model, label = item
        score = hmm_model.get_score(mfcc_features.T)
        if score > float(max_score):
            max_score = score
            output_label = label
    print(output_label)
    return output_label

if __name__ == "__main__":
    data_folder = '/app/data'
    train_wavs(data_folder)